Importazione librerie.

In [ ]:
import pandas as pd
import re
import numpy as np

Importazione data set ripuliti.

In [ ]:
spurs_c = pd.read_csv('Spurs_clean.csv')
city_c = pd.read_csv('City_clean.csv')
match_c = pd.read_csv('Match_clean.csv')

Si eliminano i tweet duplicati escludendo la colonna di attribuzione del tifo. Con la funzione drop si toglie la colonna "Unnamed: 0".

In [ ]:
tweet = pd.concat([spurs_c, city_c, match_c])
tweet = tweet.drop("Unnamed: 0", axis =1)
tweet

,user_id,screen_name,hashtags,followers,text_clean,compl_hashtags,time,minute_from_start,Moment,Min_Cronaca,compound,comp_score,tifo,Positive,Negative,Neutral
0,837362634005454848,nayupoy,"['THFC', 'MCFC', 'TOTMCI']",46,🎶 oh spurs marching,['MCFC' 'THFC' 'TOTMCI'],2021-08-15 17:30:53,0.0,Primo,0,0.0000,Neutral,Neutral,0,0,1
1,576167561,untriplesix,['COYS'],326,come boys,['COYS'],2021-08-15 17:30:53,0.0,Primo,0,0.0000,Neutral,Tottenham,0,0,1
2,518715968,NBCSportsSoccer,"['THFC', 'MCFC', 'TOTMCI']",587913,🔥 hugeeeee roar tottenham’s players come pitch...,['MCFC' 'THFC' 'TOTMCI'],2021-08-15 17:30:53,0.0,Primo,0,0.3400,Positive,Tottenham,1,0,0
3,1261979991224565760,cityscoop_,"['Mancity', 'Tottenham']",855,underway,['Mancity' 'Tottenham'],2021-08-15 17:30:54,0.0,Primo,0,0.0000,Neutral,Tottenham,0,0,1
4,266251364,lazqetjohnson,['COYS'],24053,fans best england country mile,['COYS'],2021-08-15 17:30:53,0.0,Primo,0,0.6369,Positive,Tottenham,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48393,1251407314780934144,Dj_razer_ke,['TOTMCI'],2068,live streamtottenham vs manchester citymatch l...,['TOTMCI'],2021-08-15 17:30:50,NaN,Pre,NaN,0.0000,Neutral,Else,0,0,1
48394,1302750688204795912,rizzo_x8,['TOTMCI'],13,tell you’re small club actually telling you’re...,['TOTMCI'],2021-08-15 17:30:51,NaN,Pre,NaN,0.0000,Neutral,Else,0,0,1
48395,1227274069990506497,zRadiqtion,['TOTMCI'],407,coys,['TOTMCI'],2021-08-15 17:30:51,NaN,Pre,NaN,0.0000,Neutral,Else,0,0,1
48396,1232674609377488897,___STxN,['TOTMCI'],6367,girlfriend wanting base scalp minutes game 😩 w...,['TOTMCI'],2021-08-15 17:30:52,NaN,Pre,NaN,-0.7579,Negative,Else,0,1,0


In [ ]:
tweet.dtypes

user_id                int64
screen_name           object
hashtags              object
followers              int64
text_clean            object
compl_hashtags        object
time                  object
minute_from_start    float64
Moment                object
Min_Cronaca           object
compound             float64
comp_score            object
tifo                  object
Positive               int64
Negative               int64
Neutral                int64
dtype: object

In [ ]:
tweet["user_id"] = tweet["user_id"].astype(str, copy = False)
tweet["Positive"] = tweet["Positive"].astype(bool, copy = False)
tweet["Negative"] = tweet["Negative"].astype(bool, copy = False)
tweet["Neutral"] = tweet["Neutral"].astype(bool, copy = False)
tweet.dtypes

user_id               object
screen_name           object
hashtags              object
followers              int64
text_clean            object
compl_hashtags        object
time                  object
minute_from_start    float64
Moment                object
Min_Cronaca           object
compound             float64
comp_score            object
tifo                  object
Positive                bool
Negative                bool
Neutral                 bool
dtype: object

In [ ]:
df_tweet = tweet.drop_duplicates(["user_id", "screen_name", "text_clean", "time", "compound", "Min_Cronaca"])
df_tweet

,user_id,screen_name,hashtags,followers,text_clean,compl_hashtags,time,minute_from_start,Moment,Min_Cronaca,compound,comp_score,tifo,Positive,Negative,Neutral
0,837362634005454848,nayupoy,"['THFC', 'MCFC', 'TOTMCI']",46,🎶 oh spurs marching,['MCFC' 'THFC' 'TOTMCI'],2021-08-15 17:30:53,0.0,Primo,0,0.0000,Neutral,Neutral,False,False,True
1,576167561,untriplesix,['COYS'],326,come boys,['COYS'],2021-08-15 17:30:53,0.0,Primo,0,0.0000,Neutral,Tottenham,False,False,True
2,518715968,NBCSportsSoccer,"['THFC', 'MCFC', 'TOTMCI']",587913,🔥 hugeeeee roar tottenham’s players come pitch...,['MCFC' 'THFC' 'TOTMCI'],2021-08-15 17:30:53,0.0,Primo,0,0.3400,Positive,Tottenham,True,False,False
3,1261979991224565760,cityscoop_,"['Mancity', 'Tottenham']",855,underway,['Mancity' 'Tottenham'],2021-08-15 17:30:54,0.0,Primo,0,0.0000,Neutral,Tottenham,False,False,True
4,266251364,lazqetjohnson,['COYS'],24053,fans best england country mile,['COYS'],2021-08-15 17:30:53,0.0,Primo,0,0.6369,Positive,Tottenham,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48393,1251407314780934144,Dj_razer_ke,['TOTMCI'],2068,live streamtottenham vs manchester citymatch l...,['TOTMCI'],2021-08-15 17:30:50,NaN,Pre,NaN,0.0000,Neutral,Else,False,False,True
48394,1302750688204795912,rizzo_x8,['TOTMCI'],13,tell you’re small club actually telling you’re...,['TOTMCI'],2021-08-15 17:30:51,NaN,Pre,NaN,0.0000,Neutral,Else,False,False,True
48395,1227274069990506497,zRadiqtion,['TOTMCI'],407,coys,['TOTMCI'],2021-08-15 17:30:51,NaN,Pre,NaN,0.0000,Neutral,Else,False,False,True
48396,1232674609377488897,___STxN,['TOTMCI'],6367,girlfriend wanting base scalp minutes game 😩 w...,['TOTMCI'],2021-08-15 17:30:52,NaN,Pre,NaN,-0.7579,Negative,Else,False,True,False


Check su numero valori con tifoseria "Neutral"

In [ ]:
len(df_tweet[df_tweet["tifo"] == "Neutral"])

392

Check su numero valori con tifoseria "Tottenham"

In [ ]:
len(df_tweet[df_tweet["tifo"] == "Tottenham"])

20851

Check su numero valori con tifoseria "Manchester City"

In [ ]:
len(df_tweet[df_tweet["tifo"] == "Manchester City"])

6562

Si calcolano i valori aggregati dei tweet per tifoseria e sentiment.

In [ ]:
df_tweet_tifo0 = df_tweet[["tifo", "Min_Cronaca", "compound", "Positive", "Negative", "Neutral"]]
df_tweet_tifo = df_tweet_tifo0.groupby(["Min_Cronaca","tifo"]).sum()
df_tweet_tifo

compound  Positive  Negative  Neutral
Min_Cronaca tifo                                                  
0           Else              13.7692        27        12       57
            Manchester City   16.6378        46         3       11
            Neutral            0.0000         0         0        8
            Tottenham         12.8196        32         9       51
1           Else              13.0062        35        10       55
...                               ...       ...       ...      ...
90+5        Tottenham        216.5286       429        90      398
90+6        Else             144.1138       337       114      244
            Manchester City    1.0193        50       100       28
            Neutral            0.0000         0         0        6
            Tottenham        207.6830       427        97      287

[378 rows x 4 columns]

In [ ]:
df_tweet_tifo["Total_tif"] = df_tweet_tifo["Positive"] + df_tweet_tifo["Negative"] + df_tweet_tifo["Neutral"]
df_tweet_tifo["Comp_med"] = df_tweet_tifo["compound"]/df_tweet_tifo["Positive"]
df_tweet_tifo

compound  Positive  ...  Total_tif  Comp_med
Min_Cronaca tifo                                 ...                     
0           Else              13.7692        27  ...         96  0.509970
            Manchester City   16.6378        46  ...         60  0.361691
            Neutral            0.0000         0  ...          8       NaN
            Tottenham         12.8196        32  ...         92  0.400612
1           Else              13.0062        35  ...        100  0.371606
...                               ...       ...  ...        ...       ...
90+5        Tottenham        216.5286       429  ...        917  0.504729
90+6        Else             144.1138       337  ...        695  0.427637
            Manchester City    1.0193        50  ...        178  0.020386
            Neutral            0.0000         0  ...          6       NaN
            Tottenham        207.6830       427  ...        811  0.486377

[378 rows x 6 columns]

In [ ]:
df_tweet_tifo.to_csv("Tweet_Viz.csv")

Si contano gli hashtag nelle array.

In [ ]:
df_tweet_hash_Tot = df_tweet[(df_tweet["tifo"] == "Tottenham")]
df_tweet_hash_Tot = df_tweet_hash_Tot[["compl_hashtags", "tifo"]].copy()
df_tweet_hash_MCi = df_tweet[(df_tweet["tifo"] == "Manchester City")]
df_tweet_hash_MCi = df_tweet_hash_MCi[["compl_hashtags", "tifo"]].copy()

In [ ]:
all_hash_tot = []
for row in df_tweet_hash_Tot['compl_hashtags']:
  row0 = row.replace("'","")
  row1 = row0.replace("[","")
  row2 = row1.replace("]","")
  hash = row2.split(" ")
  for i in hash:
    all_hash_tot.append(i)

all_hash_city = []
for row in df_tweet_hash_MCi['compl_hashtags']:
  row0 = row.replace("'","")
  row1 = row0.replace("[","")
  row2 = row1.replace("]","")
  hash = row2.split(" ")
  for i in hash:
    all_hash_city.append(i)


hashtag_count_Tot = {}
hashtag_count_City = {}

for hashtag in all_hash_tot:
    has=hashtag.lower()
    if has not in hashtag_count_Tot:
      hashtag_count_Tot[has] = 1
    else:
      hashtag_count_Tot[has] = hashtag_count_Tot[has] + 1

for hashtag in all_hash_city:
    has=hashtag.lower()
    if has not in hashtag_count_City:
      hashtag_count_City[has] = 1
    else:
      hashtag_count_City[has] = hashtag_count_City[has] + 1

hashtag_tot = pd.DataFrame(list(hashtag_count_Tot.items()),columns = ['Hashtag','Ricorrenza'])
hashtag_tot = hashtag_tot.sort_values(by = "Ricorrenza", ascending = False).copy()
hashtag_tot["Tifo"] = "Tottenham"

hashtag_city = pd.DataFrame(list(hashtag_count_City.items()),columns = ['Hashtag','Ricorrenza'])
hashtag_city = hashtag_city.sort_values(by = "Ricorrenza", ascending = False).copy()
hashtag_city["Tifo"] = "Manchester City"

hashtag = pd.concat([hashtag_city, hashtag_tot])
hashtag

,Hashtag,Ricorrenza,Tifo
0,mancity,4965,Manchester City
3,mcfc,1554,Manchester City
2,totmci,1163,Manchester City
16,premierleague,127,Manchester City
11,pl,125,Manchester City
...,...,...,...
378,myviking,1,Tottenham
377,hypocrisy,1,Tottenham
375,vikingstrong,1,Tottenham
368,romero,1,Tottenham


In [ ]:
hashtag.to_csv("Hashtags.csv")